## Top line analysis


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline


In [2]:
df_demo= pd.read_csv('../data/cleaned/final_demo_cleaned.csv')  
df_demo

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6,73,60,Unknown,2,45105.30,6,9
1,2304905,7,94,58,Unknown,2,110860.30,6,9
2,1439522,5,64,32,Unknown,2,52467.79,6,9
3,1562045,16,198,49,Male,2,67454.65,3,6
4,5126305,12,145,33,Female,2,103671.75,0,3
...,...,...,...,...,...,...,...,...,...
70604,7993686,4,56,38,Unknown,3,1411062.68,5,5
70605,8981690,12,148,31,Male,2,101867.07,6,6
70606,333913,16,198,61,Female,2,40745.00,3,3
70607,1573142,21,255,68,Male,3,475114.69,4,4


In [3]:
df_demo.columns


Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

## Client behavior analysis
**Who are the primary clients using this online process?**

**Assumptions** : 
- we only consider the variation Test - Control as it indicates that a client was part of the experiment after than UI changes and provides insights into how the changes affect client behavior.
- Moving forward we determined if there are commonalities among the clients who enganged in the process. 
- we conduct AB testing Test vs Control to better analyze the customer experience fully



In [4]:
df_web = pd.read_csv("../data/cleaned/Concat_Web_Data_cleaned.csv", sep=",")
df_exp = pd.read_csv("../data/cleaned/experiment_clean.csv", sep=",")

df_web, df_exp

FileNotFoundError: [Errno 2] No such file or directory: '../data/cleaned/experiment_clean.csv'

In [ ]:
#inner join to merge two tables to identify target customers 

df_web_exp = df_web.merge(df_exp[['client_id', 'variation']],
    on='client_id',
    how='inner'
)

df_web_exp

In [ ]:
# focus on Test and Control, hence disregard NaN in variation column for data accuracy

#Create a copy of the DataFrame 
df_Test_Control = df_web_exp.copy()

# Remove NaN values from the entire DataFrame
df_Test_Control.dropna(inplace=True)

print(df_Test_Control['variation'].value_counts(dropna=False))

In [ ]:
df_Test_Control

In [ ]:
# merge df_Test_Control (excl.NaN) and df_demo in order to have a more accurate customer experience dataframe

df_Total = df_Test_Control.merge(
    df_demo[['client_id','clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth']],
    on="client_id",
    how="inner"
)

df_Total

## Client behavior analysis
**Are the primary clients younger or older, new or long-standing?**

In [ ]:
df_Total['clnt_tenure_yr'].unique()

In [ ]:
df_Total['clnt_tenure_yr'].min()

In [ ]:
df_Total['clnt_tenure_yr'].max()

In [ ]:
# Relationship break down : new or long lasting relationships?

# Define the containers with bins to create tenure categories 

bins = [-1, 3, 9, 19, df_Total['clnt_tenure_yr'].max()]
labels = ["New (0–3)", 
          "Emerging (4–9)", 
          "Established (10–19)", 
          "Long-standing (20+)"
]

df_Total["tenure_group"] = pd.cut(df_Total['clnt_tenure_yr'], bins=bins, labels=labels)

tenure_summary = (df_Total["tenure_group"].value_counts().sort_index().to_frame("count"))

tenure_summary["percentage"] = (
    tenure_summary["count"] / tenure_summary["count"].sum() * 100
).round(2)

print(tenure_summary)

In [ ]:
#checking the age of the clients 

df_Total['clnt_age'].describe()

In [ ]:
df_Total['clnt_age'].unique()

In [ ]:
# new column 'age category' clustering different age groups

df_Total['age_category'] = np.select(
    [
        df_Total['clnt_age'] <= 19,
        df_Total['clnt_age'].between(20, 39),
        df_Total['clnt_age'].between(40, 64),
        df_Total['clnt_age'] >= 65
    ],
    ['Young', 'Adult', 'Middle Adult', 'Old'],
    default='Unknown')

df_Total

In [ ]:
df_Total['process_step'].unique()

In [ ]:
df_Total['age_category'].value_counts()


In [ ]:
df_Total['age_category'].value_counts(normalize=True) * 100

In [ ]:
# high level gender break down 

gender_counts = df_Total['gendr'].value_counts()

In [ ]:
# gender break down focused on Middle adult category ( 50% of age_category)

df_Total[df_Total['age_category'] == 'Middle Adult']['gendr'].value_counts()

In [ ]:
df_Total[df_Total['age_category'] == 'Middle Adult']['gendr'].value_counts(normalize = True)*100

In [ ]:

# Bar chart combining age_category and gender 



age_categories = df_Total['age_category'].unique()

# Calculate total counts for each age category
age_totals = np.array([df_Total[df_Total['age_category'] == age].shape[0] for age in age_categories])

# Calculate percentages for each gender
gender_percentages = {
    'Male': np.array([df_Total[(df_Total['age_category'] == age) & (df_Total['gendr'] == 'Male')].shape[0] / total * 100 for age, total in zip(age_categories, age_totals)]),
    'Female': np.array([df_Total[(df_Total['age_category'] == age) & (df_Total['gendr'] == 'Female')].shape[0] / total * 100 for age, total in zip(age_categories, age_totals)]),
    'Unknown': np.array([df_Total[(df_Total['age_category'] == age) & (df_Total['gendr'] == 'Unknown')].shape[0] / total * 100 for age, total in zip(age_categories, age_totals)]),
    'Non-Binary': np.array([df_Total[(df_Total['age_category'] == age) & (df_Total['gendr'] == 'Non-binary')].shape[0] / total * 100 for age, total in zip(age_categories, age_totals)])
}

width = 0.6  # width of the bars

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
bottom = np.zeros(len(age_categories))

for gender, percentages in gender_percentages.items():
    p = ax.bar(age_categories, percentages, width, label=gender, bottom=bottom)
    bottom += percentages
    ax.bar_label(p, label_type='center', fmt='%.1f%%')  # Display as percentage

ax.set_xlabel('Age Category')
ax.set_ylabel('Percentage')
ax.set_title('Gender Distribution by Age Category (Percentage)')
ax.legend()

plt.show()

## KPIs analysis 


**Engagement metric**

- Measure user engagement through calls_6_mnth and logons_6_mnth to see if the new design encourages more interactions or communication.
- calls_6_mnth: Records the number of times the client reached out over a call in the past six months.
- logons_6_mnth: Reflects the frequency with which the client logged onto Vanguard’s platform over the last six months.




In [ ]:
#Engagement KPI based on critical steps Test vs Control 


# Group by variation and process step, aggregating by client_id
engagement_summary = df_Total.groupby(['variation', 'process_step'], as_index=False).agg({
    'client_id': pd.Series.nunique # Counting unique clients
})


In [ ]:
print(engagement_summary)

In [ ]:
for variation in engagement_summary['variation'].unique():
    print(f"\nEngagement summary for {variation} group:")
    subset = engagement_summary[engagement_summary['variation'] == variation]
    for _, row in subset.iterrows():
        print(f"  {row['process_step']}: {row['client_id']} unique clients engaged")

**Overall Participation: The test group shows higher engagement across all steps than the control group. This likely indicates that the changes introduced in the test variant positively influenced user engagement**

**The "confirm" step at the end reflects higher engagement in the test group, suggesting that more users complete the entire process in the test variant compared to the control variant**


In [ ]:
# Group by 'variation' and 'process_step', and aggregate the calls and logons

engagement_summary3 = df_Total.groupby(['variation', 'process_step']).agg({
    'calls_6_mnth': 'mean',  # Use 'sum' for total, 'mean' gives average per user
    'logons_6_mnth': 'mean'
})

In [ ]:
print(engagement_summary3)


In [ ]:
engagement_summary2 = df_Total.groupby(['variation']).agg({
    'calls_6_mnth': 'mean',  # Use 'sum' for total, 'mean' gives average per user
    'logons_6_mnth': 'mean'
})

print(engagement_summary2)

**Conclusion: High level the mean of call and logons in Test vs Control decreased. I assume the new design is more intuitive, users might find it easier to navigate and complete tasks without needing as much support or multiple logins**

**Retention Rate metric**
- A/B testing: we identify which version encourages users to return and engage more.

In [ ]:
# Fix time period set up: start date vs last visit date

df_Total['date_time'].dtype

In [ ]:

df_Total['date_time'] = pd.to_datetime(df_Total['date_time'])  # Convert to datetime
df_Total['visit_date'] = df_Total['date_time'].dt.date  # Extract the date part


print(df_Total['visit_date'])

In [ ]:
cohort_period_start = pd.to_datetime('2017-04-17')
cohort_period_end = pd.to_datetime('2017-05-10')
assessment_period_end = pd.to_datetime('2017-05-31')  # personal assumption

In [ ]:
results = []

for variation in df_Total['variation'].unique():
    # Filter for initial cohort visits only
    initial_cohort = df_Total[(df_Total['variation'] == variation) &
                              (df_Total['date_time'] >= cohort_period_start) &
                              (df_Total['date_time'] <= cohort_period_end)]
    
    total_visitors = initial_cohort['client_id'].nunique()

    # Filter for return users (after initial visit and before assessment period end)
    subsequent_visits = df_Total[(df_Total['client_id'].isin(initial_cohort['client_id'])) &
                                 (df_Total['date_time'] > cohort_period_end) &
                                 (df_Total['date_time'] <= assessment_period_end)]
    
    returning_users = subsequent_visits['client_id'].nunique()

    retention_rate = (returning_users / total_visitors * 100) if total_visitors > 0 else 0
    results.append((variation, retention_rate))

for variation, rate in results:
    print(f"Retention Rate for {variation}: {rate:.2f}%")
    

**Conclusion: both engagement and retention rate in Test decreased vs Control variation in a time frame of 1 month**

## Hypothesis: "There is a significant decrease in retention due to factor X in the redesign influencing user drop-off

**Example Hypotheses and Analysis Approach**

- Hypothesis 1: "The reduction in calls and logons for the test group is due to increased efficiency, and task completion time is shorter."

Approach: Analyze and compare time spent on key actions or process steps for users in both groups.
- Hypothesis 2: "The test version's new features are causing confusion, leading to decreased retention and engagement."

Approach: Gather feedback or survey responses about new features and analyze usage patterns.
- Hypothesis 3: "The test redesign impacts different user segments differently (e.g., younger users find it easier than older users)."

Approach: Perform segmentation analysis to see if demographic characteristics influence the observed results.

In [ ]:
df_Total